In [1]:
import os
import requests
from dotenv import load_dotenv
load_dotenv()

True

### Let's implement and test the simplest flow
1. User asks a question
2. Agent decides which tool to use
3. Agent uses the tool
4. Agent returns the answer to the user

### Let's define the tools

In [2]:
check_weather_tool = {
            "name": "get_weather",
            "description": "Get the current weather in a given location",
            "input_schema": {
                "type": "object",
                "properties": {
                    "latitude": {
                        "type": "string",
                        "description": "latitude of the location for which to get the weather",
                    },
                    "longitude": {
                        "type": "string",
                        "description": "longitude of the location for which to get the weather",
                    },
                },
                "required": ["latitude", "longitude"],
            },
        }

tools = [check_weather_tool]

### Let's define the user's message

In [3]:
user_message = "What's the weather in Warsaw?"

### Let's make the first LLM call to decide which tool to use and with what parameters

In [4]:
url = "https://api.anthropic.com/v1/messages"

headers = {
    "Content-Type": "application/json",
    "x-api-key": os.getenv("ANTHROPIC_API_KEY"),
    "anthropic-version": "2023-06-01",
}

payload = {
    "model": "claude-sonnet-4-5",
    "max_tokens": 300,
    "tools": tools,
    "tool_choice": {"type": "auto"},
    "messages": [
        {
            "role": "user",
            "content": user_message,
        }
    ]
}
resp = requests.post(url, headers=headers, json=payload)

In [5]:
response_content = resp.json().get("content", [])

In [6]:
response_content

[{'type': 'text',
  'text': "I'll get the current weather in Warsaw for you. Let me fetch that information using Warsaw's coordinates (approximately 52.23°N, 21.01°E)."},
 {'type': 'tool_use',
  'id': 'toolu_012wnTRyQp6EkEfgEc5ZMCvD',
  'name': 'get_weather',
  'input': {'latitude': '52.23', 'longitude': '21.01'}}]

### Now let's process the output from the tool use step

In [7]:
# Extracting tools
tools = [block for block in response_content if block.get("type") == "tool_use"]

In [126]:
# Use tools
for tool in tools:
    if tool["name"] == "get_weather":
        latitude = tool["input"]["latitude"]
        longitude = tool["input"]["longitude"]
        weather_tool_response = requests.get(
            url="https://api.open-meteo.com/v1/forecast",
            params={"latitude": latitude, "longitude": longitude, "current_weather": True}
        )
        weather_info = weather_tool_response.json()["current_weather"]
        tool["output"] = weather_info

In [127]:
# Collecting outputs from all tools into a single string
tool_outputs = "\n".join([f"Tool: {tool['name']}, Output: {tool.get('output', 'No output')}" for tool in tools])

In [128]:
tool_outputs

"Tool: get_weather, Output: {'time': '2025-10-14T18:30', 'interval': 900, 'temperature': 15.6, 'windspeed': 12.4, 'winddirection': 188, 'is_day': 0, 'weathercode': 2}"

### Finally, let's make the last LLM call to get the final answer

In [129]:
final_prompt = f"""You are an assistant that responds to user question based on the tools used. Respond politely to user's question using the output from the tools. Use Celsius scale. If the tools output is empty, then always respond with "I don't know".

<user question>
{user_message}
</user question>

<tool_outputs>
{tool_outputs}
</tool_outputs>"""

In [130]:
print(final_prompt)

You are an assistant that responds to user question based on the tools used. Respond politely to user's question using the output from the tools. Use Celsius scale. If the tools output is empty, then always respond with "I don't know".

<user question>
What's the weather in Cape Town now?
</user question>

<tool_outputs>
Tool: get_weather, Output: {'time': '2025-10-14T18:30', 'interval': 900, 'temperature': 15.6, 'windspeed': 12.4, 'winddirection': 188, 'is_day': 0, 'weathercode': 2}
</tool_outputs>


In [131]:
payload = {
    "model": "claude-sonnet-4-5",
    "max_tokens": 300,
    "messages": [
        {
            "role": "user",
            "content": final_prompt,
        }
    ]
}
final_response = requests.post(url, headers=headers, json=payload)

### Odczytujemy finalną odpowiedź

In [132]:
print(final_response.json().get("content", [])[0].get("text", "No response"))

Based on the current weather in Cape Town:

**Temperature:** 15.6°C  
**Wind:** 12.4 km/h from the south  
**Conditions:** Partly cloudy  
**Time:** 6:30 PM (nighttime)

It's a pleasant evening in Cape Town with mild temperatures and moderate winds.
